In [11]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import numpy as np
from IPython.display import Markdown, display


In [ ]:
# from huggingface_hub import login
# login(token="<>hugging_face_token</>")  # Uncomment this line if you need to log in using Hugging Face CLI

# LOGGED IN USING HUNNGINGFACE CLI

In [2]:
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.13it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use mps:0


In [4]:
clinical_note = """
Chief Complaint:
The patient presents with abdominal pain localized to the lower right quadrant, nausea, and low-grade fever for the past 24 hours.

History & Symptoms:
- Pain began near the umbilicus and migrated to the lower right abdomen
- Mild nausea, no vomiting
- Pain increases with movement or coughing
- Temperature: 38.1°C (100.6°F)

Physical Exam:
- Rebound tenderness in the right lower quadrant
- Positive Rovsing’s sign
- No palpable masses

Diagnosis:
- Acute uncomplicated appendicitis

Plan:
- Schedule for laparoscopic appendectomy
- Start IV fluids and antibiotics pre-op
"""

In [ ]:
system_prompt = f"""
You are an expert clinical coder. From the following medical note, 
identify the most relevant ICD-10 codes:

Medical Note:
{clinical_note}

Instructions:
- Include all codes for all sections of the clinical note 
(complain, history and symptomps, diagnosis, plan
- Include a code only once

Output format:
- **Code**: <code>, **Description**: <description>

Just output the ICD-10 code and description, in the format above mentioned.
"""

In [ ]:
dummy_image = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))



In [12]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": clinical_note},
             {"type": "image", "image": dummy_image}
            
        ]
    }
]

output = pipe(text=messages, max_new_tokens=512)
display(Markdown(output[0]["generated_text"][-1]["content"]))

**Code**: K36.9, **Description**: Acute appendicitis, unspecified

**Code**: R65.9, **Description**: Nausea, unspecified

**Code**: R50.9, **Description**: Fever, unspecified

**Code**: N39.0, **Description**: Rebound tenderness

**Code**: N39.1, **Description**: Rovsing's sign

**Code**: Z98.1, **Description**: Laparoscopic appendectomy
